In [1]:
import torch
import numpy as np
import torch.nn.functional as F

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
cd '/home/abhishekshar/hierarchichal_point_process/src/'

/home/abhishekshar/hierarchichal_point_process/src


In [4]:
from hrmtpp_exact import hrmtpp_exact
from rmtpp import rmtpp

In [5]:
model = rmtpp(marker_dim=217, marker_type='binary').to(device)

In [6]:
from utils.data_loader import load_data

mimic_x, mimic_t = load_data('../data/mimic_train.pkl')
(mimic_t[8])

array([[6.42523333e+04, 2.16975694e+00, 6.42523333e+04],
       [6.46145437e+04, 6.06405093e+00, 3.62210417e+02],
       [6.46283021e+04, 1.00356481e+00, 1.37583333e+01]])

In [7]:
mimic_t[:3]

[array([[6.38807132e+04, 3.03719907e+00, 6.38807132e+04],
        [6.41642389e+04, 6.17858796e-01, 2.83525694e+02],
        [6.41656660e+04, 9.09768519e-01, 1.42708333e+00],
        [6.41701826e+04, 1.83200231e+00, 4.51666667e+00]]),
 array([[5.57389674e+04, 6.44548611e-01, 5.57389674e+04],
        [5.58828083e+04, 1.15402778e+00, 1.43840972e+02],
        [5.59125486e+04, 1.01457176e+00, 2.97402778e+01]]),
 array([[6.52360236e+04, 3.86074769e+01, 6.52360236e+04],
        [6.52775687e+04, 5.20629630e+00, 4.15451389e+01],
        [6.53220715e+04, 2.67300926e+00, 4.45027778e+01]])]

In [8]:
#First time value is the offset. High value. Removing that. Time interval column  is the last one.
t = torch.from_numpy(np.stack([mimic_t[i][:2,:] for i in range(5)], axis=1)).float().to(device)[:,:,[0,2]]
x = torch.from_numpy(np.stack([mimic_x[i][:2,:] for i in range(5)], axis=1)).float().to(device)
x.shape, t.shape, t

(torch.Size([2, 5, 217]),
 torch.Size([2, 5, 2]),
 tensor([[[6.3881e+04, 6.3881e+04],
          [5.5739e+04, 5.5739e+04],
          [6.5236e+04, 6.5236e+04],
          [7.0689e+04, 7.0689e+04],
          [8.0423e+04, 8.0423e+04]],
 
         [[6.4164e+04, 2.8353e+02],
          [5.5883e+04, 1.4384e+02],
          [6.5278e+04, 4.1545e+01],
          [7.0712e+04, 2.2917e+01],
          [8.0580e+04, 1.5665e+02]]], device='cuda:0'))

In [61]:
def compute_marker_metrics(model, x, t, mask=None):
    """
        Compute marker MSE or accuracy.
        Input: 
            model: Torch.nn module
            x : Torch of shape TxBSxmarker_dim
            t : Torch of shape TxBSx2. Last column is indexed by [actual time point, interval]
            mask is needed here.
        Output:
            y : Torch of shape TxBS
    """
    with torch.no_grad():
        h_embedded = model.compute_hidden_states(x,t,mask)
        marker_out_mu, marker_out_logvar = model.generate_marker(h_embedded, t)

        if model.marker_type == 'real':
            metric = F.mse_loss(marker_out_mu, x, reduction='none').sum(dim=-1)
        else:
            # Option 1: Compute one accuracy number for every time
            metric = (x == (marker_out_mu > 0.5).float()).float().mean(dim=-1)
            
    return metric

In [62]:
compute_marker_metrics(model, x, t)

tensor([[0.5023, 0.5115, 0.5115, 0.5115, 0.5207],
        [0.5392, 0.5161, 0.5069, 0.5023, 0.5115]], device='cuda:0')

In [48]:
%%timeit
compute_marker_metrics(model, x, t)

1.14 ms ± 54.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
def compute_point_log_likelihood(h, d_js):
        """
            Input:
                h : Tensor of shape TxBSxCx self.shared_output_layers[-1]
                tj : Tensor of shape TxBSx2 [j,:,0] represents actual time at timestep j ,\
                    [i,:,1] represents time gap d_i = t_i- t_{i-1}
                d_js: Tensor of shape TxBSx1x(N+1) [j,:,:,k] represents the k*delta_x in trapezoidal rule
                [j,:,:,0] is 0
                https://en.wikipedia.org/wiki/Trapezoidal_rule#Uniform_grid
            Output:
                log_f_t : tensor of shape TxBSxC

        """
        #hz_embedded shape TxBSx64, d_js shape is TxBSx1xN
        
        
        #d_js = tj[:, :, 0:1, None]  # Shape TxBSx1xN Time differences

        past_influence = model.h_influence(h)[:,:,:,None]  # TxBSx1,x1


        # TxBSx1x1
        current_influence = model.time_influence * d_js#TxBSx1xN
        base_intensity = model.base_intensity[:,:,:, None]  # 1x1x1x1
        #print(past_influence.shape,current_influence.shape, base_intensity.shape)

        term1 = past_influence + current_influence + base_intensity
        term2 = (past_influence + base_intensity).exp()
        term3 = term1.exp()

        log_f_t = term1 + \
            (1./(model.time_influence+1e-6 )) * (term2-term3)
        return log_f_t  # TxBSxCx(N+1)

In [38]:
marker_out_mu.shape, metric.shape

(torch.Size([2, 5, 217]), torch.Size([2, 5]))

In [39]:
metric

tensor([[0.5023, 0.5115, 0.5115, 0.5115, 0.5207],
        [0.5392, 0.5161, 0.5069, 0.5023, 0.5115]], device='cuda:0')

In [48]:
times.shape, g.shape, l.sum(dim=-1), delta_x

NameError: name 'times' is not defined

In [40]:
integral.shape

torch.Size([3, 2, 5, 1])

In [41]:
time_log_likelihood

tensor([[[[ 7.4486e-03, -1.0391e+02, -3.1650e+02,  ..., -2.0832e+32,
           -4.2468e+32, -8.6578e+32],
          [-3.6657e-02, -9.9441e+01, -3.0283e+02,  ..., -1.9933e+32,
           -4.0636e+32, -8.2842e+32],
          [-2.2586e-01, -8.2372e+01, -2.5058e+02,  ..., -1.6497e+32,
           -3.3631e+32, -6.8562e+32],
          [ 5.7212e-02, -1.0920e+02, -3.3268e+02,  ..., -2.1895e+32,
           -4.4635e+32, -9.0996e+32],
          [-5.7042e-02, -9.7441e+01, -2.9671e+02,  ..., -1.9531e+32,
           -3.9816e+32, -8.1171e+32]],

         [[ 7.4486e-03, -1.0391e+02, -3.1650e+02,  ..., -2.0830e+32,
           -4.2465e+32, -8.6571e+32],
          [-3.6657e-02, -9.9441e+01, -3.0283e+02,  ..., -1.9931e+32,
           -4.0633e+32, -8.2836e+32],
          [-2.2586e-01, -8.2372e+01, -2.5058e+02,  ..., -1.6496e+32,
           -3.3629e+32, -6.8557e+32],
          [ 5.7212e-02, -1.0920e+02, -3.3268e+02,  ..., -2.1893e+32,
           -4.4632e+32, -9.0989e+32],
          [-5.7042e-02, -9.7441e+01

In [26]:
t

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1549635019666/work/aten/src/THC/generated/../THCTensorMathCompareT.cuh:69

In [43]:
dmax

tensor(71228.6328, device='cuda:0')